# Amanogawa Notebook 01: Band Analysis (Thin Demo)

This notebook is a lightweight demo. The core implementation lives in `src/amanogawa/`.
No scientific core logic is reimplemented here.

- Purpose: quick web/Colab trial
- Source of truth: package modules under `amanogawa.*`
- Outputs: `outputs/notebooks/01_band_analysis/`


In [ ]:
from __future__ import annotations

import json
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

ROOT_CANDIDATES = [Path("..").resolve(), Path(".").resolve()]
ROOT = next((p for p in ROOT_CANDIDATES if (p / "src" / "amanogawa").exists()), ROOT_CANDIDATES[0])
SRC_DIR = ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

from amanogawa import __version__
from amanogawa.band_geometry import analyze_band_geometry
from amanogawa.detection import detect_stars_log, plot_star_overlay, threshold_sweep
from amanogawa.io import load_image_gray
from amanogawa.photometry import assign_magnitude_bins_from_files
from amanogawa.spatial_stats import (
    boxcount_fractal_dimension,
    nearest_neighbor_distances,
    two_point_correlation_function,
)

IMAGE_PATH = ROOT / "data" / "raw" / "IMG_5991.jpg"
OUT_DIR = ROOT / "outputs" / "notebooks" / "01_band_analysis"
DETECTION_DIR = OUT_DIR / "detection"
STATS_DIR = OUT_DIR / "spatial_stats"
BAND_DIR = OUT_DIR / "band_geometry"
PHOT_DIR = OUT_DIR / "photometry"

for p in [DETECTION_DIR, STATS_DIR, BAND_DIR, PHOT_DIR]:
    p.mkdir(parents=True, exist_ok=True)

SEED = 42
THRESHOLD = 0.05
MAX_SIGMA = 6.0
NUM_SIGMA = 12

print(f"Amanogawa version: {__version__}")
print(f"Image: {IMAGE_PATH}")
print(f"Output root: {OUT_DIR}")


In [ ]:
image, (width_px, height_px) = load_image_gray(IMAGE_PATH)
coords_df = detect_stars_log(
    image,
    threshold=THRESHOLD,
    max_sigma=MAX_SIGMA,
    num_sigma=NUM_SIGMA,
)

coords_path = DETECTION_DIR / "star_coords.csv"
coords_df.to_csv(coords_path, index=False)

summary = {
    "status": "ok",
    "image": IMAGE_PATH.name,
    "image_path": str(IMAGE_PATH),
    "width_px": int(width_px),
    "height_px": int(height_px),
    "num_stars": int(len(coords_df)),
    "coords_csv": str(coords_path),
    "detection": {
        "threshold": float(THRESHOLD),
        "max_sigma": float(MAX_SIGMA),
        "num_sigma": int(NUM_SIGMA),
    },
}
(DETECTION_DIR / "detection_summary.json").write_text(json.dumps(summary, indent=2), encoding="utf-8")

plot_star_overlay(IMAGE_PATH, coords_df, DETECTION_DIR / "star_detection_overlay.png")

thresholds = np.linspace(0.03, 0.08, 10)
threshold_sweep(
    IMAGE_PATH,
    thresholds=thresholds,
    out_dir=DETECTION_DIR,
    max_sigma=MAX_SIGMA,
    num_sigma=NUM_SIGMA,
)

print(f"Detected stars: {len(coords_df)}")
coords_df.head()


In [ ]:
points = coords_df[["x", "y"]].to_numpy(dtype=float)

if len(points) == 0:
    stats_payload = {
        "status": "no_detections",
        "image_size": {"width_px": int(width_px), "height_px": int(height_px)},
        "sampling": {"seed": int(SEED), "max_points": 3500},
        "nearest_neighbor": {"mean": None, "std": None, "median": None, "min": None, "max": None},
        "fractal_dimension": None,
        "fractal_boxcount": {"eps": [], "box_counts": []},
        "two_point_correlation": {"r_centers": [], "xi_values": [], "xi_mean": None, "xi_max": None},
    }
else:
    nnd = nearest_neighbor_distances(points)
    fractal_d, eps, box_counts = boxcount_fractal_dimension(points, width_px, height_px)

    r_max = min(width_px, height_px) / 3.0
    r_bins = np.linspace(5.0, r_max, 22)
    rc, xi = two_point_correlation_function(
        points,
        width_px,
        height_px,
        r_bins,
        max_points=3500,
        rng=np.random.default_rng(SEED),
    )

    finite_nnd = nnd[np.isfinite(nnd)]
    finite_xi = xi[np.isfinite(xi)]

    stats_payload = {
        "status": "ok",
        "image_size": {"width_px": int(width_px), "height_px": int(height_px)},
        "sampling": {"seed": int(SEED), "max_points": 3500},
        "nearest_neighbor": {
            "mean": float(np.mean(finite_nnd)) if finite_nnd.size else None,
            "std": float(np.std(finite_nnd)) if finite_nnd.size else None,
            "median": float(np.median(finite_nnd)) if finite_nnd.size else None,
            "min": float(np.min(finite_nnd)) if finite_nnd.size else None,
            "max": float(np.max(finite_nnd)) if finite_nnd.size else None,
        },
        "fractal_dimension": float(fractal_d) if np.isfinite(fractal_d) else None,
        "fractal_boxcount": {"eps": eps.tolist(), "box_counts": box_counts.tolist()},
        "two_point_correlation": {
            "r_centers": rc.tolist(),
            "xi_values": xi.tolist(),
            "xi_mean": float(np.mean(finite_xi)) if finite_xi.size else None,
            "xi_max": float(np.max(finite_xi)) if finite_xi.size else None,
        },
    }

(STATS_DIR / "spatial_statistics_analysis.json").write_text(json.dumps(stats_payload, indent=2), encoding="utf-8")
stats_payload


In [ ]:
fit = analyze_band_geometry(points, width_px, height_px, bins_x=60, nbins_profile=200)
is_empty = (not np.isfinite(fit.angle_deg)) and len(fit.y_centers) == 0

band_payload = {
    "status": "no_detections" if is_empty else "ok",
    "principal_axis": {
        "angle_deg": float(fit.angle_deg) if np.isfinite(fit.angle_deg) else float("nan"),
        "center_px": [float(fit.center_px[0]), float(fit.center_px[1])],
        "axis_ratio": float(fit.axis_ratio) if np.isfinite(fit.axis_ratio) else float("nan"),
    },
    "band_width_measurements": {
        "gaussian_fwhm_px": fit.gaussian_fwhm_px,
        "lorentzian_fwhm_px": fit.lorentzian_fwhm_px,
        "empirical_fwhm_px": float(fit.empirical_fwhm_px),
    },
    "profile": {"y_centers": fit.y_centers, "density": fit.density_profile},
}
(BAND_DIR / "band_geometry_analysis.json").write_text(json.dumps(band_payload, indent=2), encoding="utf-8")

try:
    mag_path = assign_magnitude_bins_from_files(IMAGE_PATH, coords_path, PHOT_DIR, seed=SEED)
    print(f"Magnitude bins CSV: {mag_path}")
except Exception as exc:
    print(f"Photometry step skipped: {exc}")

band_payload["status"]


In [ ]:
generated_files = sorted(str(p.relative_to(ROOT)) for p in OUT_DIR.rglob("*") if p.is_file())
print("Generated files:")
for p in generated_files:
    print(f" - {p}")

fig, ax = plt.subplots(figsize=(8, 3))
ax.axis("off")
ax.text(0.0, 1.0, "Notebook 01 complete", fontsize=14, va="top")
ax.text(0.0, 0.75, f"Detected stars: {len(coords_df)}", fontsize=11, va="top")
ax.text(0.0, 0.55, f"Output dir: {OUT_DIR}", fontsize=10, va="top")
plt.show()
